In [2]:
import pandas as pd

crunch = pd.read_csv("crunchbase-investments.csv",engine='python')
crunch.head()

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012.0,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012.0,20000.0
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012.0,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012.0,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011.0,20000.0


In [3]:
"""Evaluate dataframe chunks"""

# These are the important columns we need for analysis. The 
# rest do not need to be brought in.
cols = ['company_name','company_category_code','company_city',
        'investor_name','investor_category_code','investor_city',
        'funding_round_type','funded_at','funded_month',
        'funded_year','funded_quarter','raised_amount_usd']

# Read csv file in as a dataframe in chunks
chunk_iter = pd.read_csv("crunchbase-investments.csv",
                        chunksize=5000,usecols=cols,
                        engine='python')

total_mem = 0

# Calculate total memory usage across all chunks
for chunk in chunk_iter:
    total_mem += chunk.memory_usage(deep=True).sum()
print("Total Memory Usage:",total_mem/(1024*1024),"MB\n")

# Evaluate each column
for col in chunk:
    
    # number of missing values
    null_count = chunk[col].isnull().sum()
    
    # memory usage
    col_mem = chunk[col].memory_usage(deep=True)
    
    # datatype
    col_type = chunk[col].dtypes
    
    # Percent of values that are unique
    pct_uniq = (chunk[col].nunique() / len(chunk[col]))*100
    
    print(col,"-\n\t",col_type,"\n\t",null_count,"missing values\n\t",
          ("%.0f"%pct_uniq),"% unique values\n\t",
          ("%.2f"%(col_mem/(1024*1024))),"MB")

Total Memory Usage: 31.005454063415527 MB

company_name -
	 object 
	 0 missing values
	 54 % unique values
	 0.18 MB
company_category_code -
	 object 
	 10 missing values
	 1 % unique values
	 0.18 MB
company_city -
	 object 
	 45 missing values
	 9 % unique values
	 0.18 MB
investor_name -
	 object 
	 0 missing values
	 47 % unique values
	 0.19 MB
investor_category_code -
	 float64 
	 2870 missing values
	 0 % unique values
	 0.02 MB
investor_city -
	 float64 
	 2870 missing values
	 0 % unique values
	 0.02 MB
funding_round_type -
	 object 
	 0 missing values
	 0 % unique values
	 0.17 MB
funded_at -
	 object 
	 0 missing values
	 31 % unique values
	 0.18 MB
funded_month -
	 object 
	 0 missing values
	 4 % unique values
	 0.18 MB
funded_quarter -
	 object 
	 0 missing values
	 2 % unique values
	 0.18 MB
funded_year -
	 int64 
	 0 missing values
	 1 % unique values
	 0.02 MB
raised_amount_usd -
	 float64 
	 290 missing values
	 12 % unique values
	 0.02 MB


In [4]:
"""Change column datatypes to reduce memory usage"""
"""Connect to and export data to SQLite database"""

# Connect to SQLite database
import sqlite3
conn = sqlite3.connect('crunchbase.db')

# Read csv file in as a dataframe in chunks
chunk_iter = pd.read_csv("crunchbase-investments.csv",
                        chunksize=5000,usecols=cols,
                        engine='python')

total_mem = 0


for chunk in chunk_iter:
    
    # convert to categorical
    chunk['company_category_code']=chunk['company_category_code'].astype('category')
    chunk['company_city']=chunk['company_city'].astype('category')
    chunk['investor_category_code']=chunk['investor_category_code'].astype('category')
    chunk['investor_city']=chunk['investor_city'].astype('category')
    chunk['funding_round_type']=chunk['funding_round_type'].astype('category')
    
    # convert to datetime
    chunk['funded_at']=pd.to_datetime(chunk['funded_at'],format='%Y-%m-%d',errors='coerce')
    chunk['funded_year']=pd.to_datetime(chunk['funded_year'],format='%Y',errors='coerce')
    
    # clean data then convert to integer (for months, no missing values)
    chunk['funded_month'] = chunk['funded_month'].str.split('-').str[1]
    chunk['funded_month'] = pd.to_datetime(chunk['funded_month'],format='%m',errors='coerce')
    
    # clean data then convert to categorical
    chunk['funded_quarter'] = chunk['funded_quarter'].str.split('-').str[1]
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('category')
    
    # Calculate total memory usage across all chunks
    total_mem += chunk.memory_usage(deep=True).sum()
    
    # Export each chunk to a new table in SQLite database
    chunk.to_sql("investments", conn, if_exists='append', index=False)
    
    
print("Total Memory Usage:",total_mem/(1024*1024),"MB\n")

Total Memory Usage: 9.96079158782959 MB



In [5]:
"""Query table"""
cur = conn.cursor()
cur.execute("SELECT * FROM investments LIMIT 10;")
rows = cur.fetchall()
for row in rows[:10]:
    print(row)

('AdverCar', 'advertising', 'San Francisco', '1-800-FLOWERS.COM', None, 'New York', 'series-a', '2012-10-30 00:00:00', '1900-10-01 00:00:00', 'Q4', '2012-01-01 00:00:00', 2000000.0)
('LaunchGram', 'news', 'Mountain View', '10Xelerator', 'finance', 'Columbus', 'other', '2012-01-23 00:00:00', '1900-01-01 00:00:00', 'Q1', '2012-01-01 00:00:00', 20000.0)
('uTaP', 'messaging', None, '10Xelerator', 'finance', 'Columbus', 'other', '2012-01-01 00:00:00', '1900-01-01 00:00:00', 'Q1', '2012-01-01 00:00:00', 20000.0)
('ZoopShop', 'software', 'columbus', '10Xelerator', 'finance', 'Columbus', 'angel', '2012-02-15 00:00:00', '1900-02-01 00:00:00', 'Q1', '2012-01-01 00:00:00', 20000.0)
('eFuneral', 'web', 'Cleveland', '10Xelerator', 'finance', 'Columbus', 'other', '2011-09-08 00:00:00', '1900-09-01 00:00:00', 'Q3', '2011-01-01 00:00:00', 20000.0)
('Tackk', 'web', 'Cleveland', '10Xelerator', 'finance', 'Columbus', 'other', '2012-02-01 00:00:00', '1900-02-01 00:00:00', 'Q1', '2012-01-01 00:00:00', 2000

In [6]:
"""Analyze Crunchbase investments"""


"""Find Categories with the Most Investments"""
query = """SELECT company_category_code,COUNT(*) AS counts
           FROM investments
           GROUP BY company_category_code
           ORDER BY counts DESC;"""
cat_counts = pd.read_sql(query, conn)
print(cat_counts[:10])

  company_category_code  counts
0              software   28972
1                   web   20060
2               biotech   19804
3            enterprise   17956
4                mobile   16268
5           advertising   12800
6             ecommerce    8672
7             cleantech    7792
8           games_video    7572
9             analytics    7452


In [7]:
"""Investor that contributed the most money"""

query = """SELECT investor_name,SUM(raised_amount_usd) AS money
           FROM investments
           GROUP BY investor_name
           ORDER BY money DESC;"""
investor_counts = pd.read_sql(query, conn)
print(investor_counts[:10])

                      investor_name         money
0  Kleiner Perkins Caufield & Byers  4.487131e+10
1         New Enterprise Associates  3.877017e+10
2                    Accel Partners  2.588850e+10
3                     Goldman Sachs  2.550184e+10
4                   Sequoia Capital  2.415761e+10
5                             Intel  2.387680e+10
6                            Google  2.323520e+10
7                       Time Warner  2.292000e+10
8                           Comcast  2.267600e+10
9                 Greylock Partners  1.984393e+10


In [8]:
"""Investors that contributed the most money per startup"""

query = """SELECT investor_name,SUM(raised_amount_usd) AS money,company_name
           FROM investments
           GROUP BY investor_name,company_name
           ORDER BY money DESC;"""
fund_counts = pd.read_sql(query, conn)
print(fund_counts[:10])

              investor_name         money company_name
0                   Comcast  2.248000e+10    Clearwire
1                     Intel  2.248000e+10    Clearwire
2               Time Warner  2.248000e+10    Clearwire
3               BrightHouse  1.880000e+10    Clearwire
4                    Google  1.280000e+10    Clearwire
5    Marlin Equity Partners  1.040000e+10    sigmacare
6             Sprint Nextel  1.000000e+10    Clearwire
7      Eagle River Holdings  9.680000e+09    Clearwire
8  Digital Sky Technologies  6.800000e+09     Facebook
9             Goldman Sachs  6.000000e+09     Facebook


In [9]:
conn.close()